In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')
import os
os.environ['PATH'] = '/Users/ravi.tej/anaconda3/envs/bertopicenv/bin:/Users/ravi.tej/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin'

In [2]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [17]:
!pip install fuzzywuzzy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [54]:
!pip install 'fuzzywuzzy[speedup]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 kB 1.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [56]:
def get_all_symbol_names():
    with PostgresDatabaseOperation() as cursor:
        sql = 'SELECT DISTINCT symbol, name FROM stock_ticker_info'
        cursor.execute(sql)
        results = cursor.fetchall()
    
    return results

In [157]:
def get_all_company_wise_volume():
    with PostgresDatabaseOperation() as cursor:
        sql = "SELECT symbol, AVG(volume * close) FROM stock_history WHERE symbol LIKE '%.NS' AND date > '2024-01-01' GROUP BY symbol"
        cursor.execute(sql)
        results = cursor.fetchall()
    return results

In [158]:
avg_volume_details = get_all_company_wise_volume()

In [129]:
import numpy as np

In [159]:
volume_dict = {x[0]: {'abs': float(x[1]), 'log': np.log2(float(x[1]))} for x in avg_volume_details}

/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_96454/1270495777.py:1: RuntimeWarning: divide by zero encountered in log2
  volume_dict = {x[0]: {'abs': float(x[1]), 'log': np.log2(float(x[1]))} for x in avg_volume_details}


In [116]:
import pandas as pd

In [160]:
volume_df = pd.DataFrame(avg_volume_details, columns = ['symbol', 'avg_volume']).set_index('symbol').reset_index()

In [167]:
volume_df['log_volume'] = volume_df['avg_volume'].apply(lambda x: np.log2(float(x))/np.log2(1.5) if float(x) > 1 else 1)

In [168]:
volume_df

,symbol,avg_volume,log_volume
0,20MICRONS.NS,30719768.17615295508091,42.520100
1,21STCENMGM.NS,301008.914794998170304,31.112160
2,3IINFOLTD.NS,161305952.576985397372464,46.610209
3,3MINDIA.NS,160494037.199101563,46.597764
4,3PLAND.NS,2059614.987348632803598,35.855193
...,...,...,...
1799,ZODIAC.NS,5687203.40596631106407,38.360216
1800,ZOMATO.NS,9866556384.38835288761918,56.755603
1801,ZOTA.NS,18296907.87303894009319,41.242125
1802,ZUARI.NS,153157835.41733276448707,46.482371


In [23]:
symbol_names = get_all_symbol_names()

In [322]:
symbol_name_mapping = {x[0]: x[1] for x in symbol_names}

In [323]:
len(symbol_name_mapping)

5422

In [55]:
from fuzzywuzzy import process, fuzz

In [80]:
name_to_symbol = {}

In [81]:
for sym, name in symbol_names:
    if sym.split('.')[1] == 'NS':
        if name in symbol_names:
            name_to_symbol[name].append(sym)
        else:
            name_to_symbol[name] = [sym]

In [90]:
len(name_to_symbol)

1931

In [82]:
distinct_names = list(name_to_symbol.keys())

In [106]:
distinct_symbols = [x[0] for x in symbol_names if '.NS' in x[0]]

In [326]:
symbol_name_combo = []
for sym in distinct_symbols:
    symbol_name_combo.append(sym.strip('.NS') + ': ' + symbol_name_mapping[sym])

In [330]:
name_symbol_combo_mapping = {x: x.split(':')[0] for x in symbol_name_combo}

In [332]:
def get_symbol_for_name(name):
    k = process.extract(name, distinct_names)
    p = process.extract(name, distinct_symbols)
    symbols_matched = []
    for name_match, score in k:
        symbols_matched.extend([(cur_symbol,score) for cur_symbol in name_to_symbol[name_match]])
    # sym = name_to_symbol[matching_name][0]
    matches = pd.DataFrame(symbols_matched + p, columns = ['symbol', 'matching_score'])
    matches = pd.merge(matches, volume_df, how = 'left', on = 'symbol')
    matches['score'] = matches['matching_score'] * matches['log_volume']
    return matches[matches['matching_score'] > 75].nlargest(5, 'score')

In [333]:
import time

In [340]:
process.extract('SBI', distinct_symbols)

[('SBIETFQLTY.NS', 90),
 ('SBILIFE.NS', 90),
 ('SBIN.NS', 90),
 ('KREBSBIO.NS', 90),
 ('SBIETFPB.NS', 90)]

In [334]:
s = time.time()
k = get_symbol_for_name('SBI')
e = time.time()

In [335]:
k

,symbol,matching_score,avg_volume,log_volume,score


In [336]:
e - s

0.18636512756347656

In [295]:
def get_stock_prices_between_dates(symbol, time_period='3 month'):
    with PostgresDatabaseOperation() as cursor:
        sql = f"SELECT date, symbol, close FROM stock_history_latest WHERE symbol in ('{symbol}', '^NSEI') AND date >= CURRENT_DATE - INTERVAL '{time_period}'"
        cursor.execute(sql)
        results = cursor.fetchall()
    return results

def get_avg_stock_prices_between_dates(symbols, time_period='3 month'):
    with PostgresDatabaseOperation() as cursor:
        sql = f"SELECT symbol, avg(close) FROM stock_history_latest WHERE symbol in ('{symbol}', '^NSEI') AND date >= CURRENT_DATE - INTERVAL '{time_period}' GROUP BY symbol"
        cursor.execute(sql)
        results = cursor.fetchall()
    return results

def beta_calculation_query(symbols, time_period='1 year'):
    market_symbol = '^NSEI'  # NSE Index symbol
    symbol_string = ','.join(f"'{sym}'" for sym in symbols)

    stock_returns_subquery = f"""
    SELECT symbol, 
           date, 
           (close - LAG(close, 1) OVER (PARTITION BY symbol ORDER BY date)) / LAG(close, 1) OVER (PARTITION BY symbol ORDER BY date) AS daily_return
    FROM stock_history
    WHERE symbol IN ({symbol_string}) AND date >= CURRENT_DATE - INTERVAL '{time_period}'
    """

    market_returns_subquery = f"""
    SELECT date, 
           (close - LAG(close, 1) OVER (ORDER BY date)) / LAG(close, 1) OVER (ORDER BY date) AS daily_return
    FROM stock_history
    WHERE symbol = '{market_symbol}' AND date >= CURRENT_DATE - INTERVAL '{time_period}'
    """

    # Assuming you want to calculate beta for each stock symbol against the market index
    beta_query = f"""
    SELECT s.symbol, 
           (COVAR_SAMP(s.daily_return, m.daily_return) / VAR_SAMP(m.daily_return)) AS beta
    FROM ({stock_returns_subquery}) s
    INNER JOIN ({market_returns_subquery}) m ON s.date = m.date
    GROUP BY s.symbol
    """
    
    with PostgresDatabaseOperation() as cursor:
        cursor.execute(beta_query)
        results = cursor.fetchall()
    return results

In [306]:
beta_calculation_query(['TCS.NS', 'RELIANCE.NS'],time_period='10 week')

[('RELIANCE.NS', 1.4161375296355334), ('TCS.NS', 0.5823065152971731)]

In [290]:
def get_volatility(symbols, time_period='3 month', sort_order='ASC'):
    with PostgresDatabaseOperation() as cursor:
        symbols = symbols[:5]
        if symbols:
            symbol_string = ''
            for sym in symbols:
                symbol_string = symbol_string + f"'{sym}',"
            symbol_string = symbol_string.strip(',')
            sql = f"""
            SELECT symbol, STDDEV(close) AS volatility
            FROM stock_history
            WHERE date >= CURRENT_DATE - INTERVAL '{time_period}'
                  AND symbol in ({symbol_string})
            GROUP BY symbol
            ORDER BY volatility {sort_order};
            """
        else:
            sql = f"""
            SELECT symbol, STDDEV(close) AS volatility
            FROM stock_history
            WHERE date >= CURRENT_DATE - INTERVAL '{time_period}'
            GROUP BY symbol
            ORDER BY volatility {sort_order};
            """
        cursor.execute(sql)
        results = cursor.fetchall()
    return results

In [285]:
def get_stocks_price_between_dates(symbols, time_period='3 month'):
    symbol_string = ''
    for sym in symbols:
        symbol_string = symbol_string + f"'{sym}',"
    symbol_string = symbol_string.strip(',')
    with PostgresDatabaseOperation() as cursor:
        sql = f"SELECT date, symbol, close FROM stock_history_latest WHERE symbol in ({symbol_string}) AND date >= '{start_date}' AND date <= '{end_date}'"
        cursor.execute(sql)
        results = cursor.fetchall()
    return results

In [277]:
s = time.time()
k = get_stocks_price_between_dates(['TCS.NS'], "2024-01-01", "2024-03-05")
e = time.time()

In [238]:
s = time.time()
k = get_stock_NSE_details_between_dates('TCS.NS', "2024-01-01", "2024-03-05")
e = time.time()

In [291]:
k = get_volatility(['TCS.NS', '^NSEI'])

In [292]:
k

[('TCS.NS', Decimal('157.603957401251119946394018')),
 ('^NSEI', Decimal('331.969166741157182522'))]

In [231]:
k[0][0].strftime('%Y-%m-%d')

'2024-01-01'

In [241]:
kdf = pd.DataFrame(k, columns = ['date', 'symbol', 'price'])

In [261]:
kdf_pivot = pd.pivot_table(kdf, index = 'date', columns = 'symbol', values='price',aggfunc='max').reset_index()

In [262]:
kdf_pivot['TCS.NS'] = kdf_pivot['TCS.NS'].astype('float')
kdf_pivot['^NSEI'] = kdf_pivot['^NSEI'].astype('float')

In [260]:
from datetime import datetime, timedelta

def add_time_to_datetime(dt_input):
    # Add 5 hours and 30 minutes to the input datetime
    new_dt = dt_input + timedelta(hours=5, minutes=30)

    # Convert the resulting datetime back to a string
    new_date_str = new_dt.strftime('%Y-%m-%d')

    return new_date_str

# Example usage
date_str = "2024-01-01 18:30:00+00:00"
dt_input = datetime.fromisoformat(date_str)
new_date_str = add_time_to_datetime(dt_input)
print(new_date_str)

2024-01-02


In [263]:
kdf_pivot['date'] = kdf_pivot['date'].apply(lambda x: add_time_to_datetime(x))

In [264]:
kdf_pivot.to_dict('list')

{'date': ['2024-01-02',
  '2024-01-03',
  '2024-01-04',
  '2024-01-05',
  '2024-01-08',
  '2024-01-09',
  '2024-01-10',
  '2024-01-11',
  '2024-01-12',
  '2024-01-15',
  '2024-01-16',
  '2024-01-17',
  '2024-01-18',
  '2024-01-19',
  '2024-01-23',
  '2024-01-24',
  '2024-01-25',
  '2024-01-29',
  '2024-01-30',
  '2024-01-31',
  '2024-02-01',
  '2024-02-02',
  '2024-02-05',
  '2024-02-06',
  '2024-02-07',
  '2024-02-08',
  '2024-02-09',
  '2024-02-12',
  '2024-02-13',
  '2024-02-14',
  '2024-02-15',
  '2024-02-16',
  '2024-02-19',
  '2024-02-20',
  '2024-02-21',
  '2024-02-22',
  '2024-02-23',
  '2024-02-26',
  '2024-02-27',
  '2024-02-28',
  '2024-02-29',
  '2024-03-01',
  '2024-03-04',
  '2024-03-05'],
 'TCS.NS': [3783.199951171875,
  3691.75,
  3666.800048828125,
  3737.89990234375,
  3678.300048828125,
  3689.89990234375,
  3713.050048828125,
  3735.550048828125,
  3882.800048828125,
  3903.800048828125,
  3861.300048828125,
  3884.60009765625,
  3902.60009765625,
  3943.05004882812

In [268]:
kdf_pivot[['TCS.NS', '^NSEI']].mean().to_dict()

{'TCS.NS': 3946.7500277432528, '^NSEI': 21831.396528764206}

In [251]:
e - s

0.3376281261444092